In [1]:
import pandas as pd
import numpy as np
from random import shuffle



x = pd.read_csv('X.csv',header=None)
y = pd.read_csv('y.csv',header=None)
x_train = np.array(x)
y_test = np.array(y)

# Q1

In [2]:
def n_per_category(category,test,x=x):
    '''Calculating the number of data point per category'''
    x_temp = x[x[:,2]==category][:]
    cout = 0
    for i in x_temp[:,:2]:
        if np.all(i == test):
            cout = cout + 1
    return cout

def similarity(x1,y1,a = [0.6,0.4]):
    '''calcualting the similarity between the test and train points'''
    d = 0
    for i in range(len(y1)):
        d = d + a[i]*(x1[i]-y1[i])
    if d < 0:
        d = d*-1
    s = np.exp(-d)
    return s

def num(x,y,gamma,i,a):
    '''for each exmpler calculate similarity with new stimulus and multiply with exemplar's memebership count.'''
    temp = 0
    new_x = np.unique(x,axis=0)
    for j in new_x:
        temp = temp + n_per_category(i,j[:2],x)*similarity(j,y,a)
    temp = temp*gamma
    return temp


def den(x,y,gamma,a):
    '''calulating total probability'''
    n = 0
    new_x = np.unique(x,axis=0)
    for i in range(len(gamma)):
        temp = 0
        for j in new_x:
            temp = temp + n_per_category(i+1,j[:2],x)*similarity(j,y,a)
        n = n + temp*gamma[i]
    return n

def gcm(x,y,a,g):
    '''gcm funtion'''
    gamma = g
    denominator = den(x,y,gamma,a)
    cate = []
    for i in range(len(gamma)):
        cate.append(num(x,y,gamma[i],i+1,a)/denominator)
    return cate
    

def predict(x,y,a,g):
    '''predicting the value of test data'''
    pred_y = []
    for i in range(len(y)):
        pred_y.append(np.argmax(gcm(x,y[i],a,g))+1) 
    return pred_y

#value of alpha 
a = [0.6,0.4]  
#value of gamma 
g = [0.7,1.5,0.8]

print('\nPredicted value of testing example',predict(x_train,y_test,a,g) ,'for alpha -',a[0],a[1], 'and gamma -',g[0],g[1],g[2])



Predicted value of testing example [2, 2, 3, 2, 3, 3, 2, 2, 2, 1] for alpha - 0.6 0.4 and gamma - 0.7 1.5 0.8


In [3]:

a = [0.7,0.3]  
g = [0.5,1.5,1]

print('\nPredicted value of testing example',predict(x_train,y_test,a,g) ,'for alpha -',a[0],a[1], 'and gamma -',g[0],g[1],g[2])

a = [0.7,0.3]  
g = [1.3,1.5,1]
pred_y = predict(x_train,y_test,a,g)
print('\nPredicted value of testing example',predict(x_train,y_test,a,g) ,'for alpha -',a[0],a[1], 'and gamma -',g[0],g[1],g[2])


Predicted value of testing example [2, 2, 3, 2, 2, 3, 2, 2, 2, 1] for alpha - 0.7 0.3 and gamma - 0.5 1.5 1

Predicted value of testing example [2, 2, 3, 2, 2, 3, 2, 2, 2, 1] for alpha - 0.7 0.3 and gamma - 1.3 1.5 1


In [4]:
y_test = list(y_test)
y = pd.DataFrame(y_test)
y[2] = pred_y
y.to_csv('Q1_Y.csv',index= None)
y

,0,1,2
0,74,67,2
1,69,63,2
2,92,81,3
3,64,61,2
4,66,84,2
5,76,68,3
6,61,58,2
7,64,76,2
8,68,66,2
9,34,61,1


In question it is given that more likely to use weights than height so in aplha the value of weight is more than height 

And also in gamma the value of average is greater than large since he is more likely to call average than large

# Q2

In [5]:
class dLocalMAP:
    """
    See Anderson (1990, 1991)
    'Categories' renamed 'clusters' to avoid confusion.
    Discrete version.
    
    Stimulus format is a list of integers from 0 to n-1 where n is the number
    of possible features (e.g. [1,0,1])
    
    args: c, alphas
    """
    
    def __init__(self, args):
        self.partition = [[]]
        self.c, self.alpha = args
        self.alpha0 = [sum(self.alpha[0]),sum(self.alpha[1]),sum(self.alpha[2])]
        self.N = 0
    
    def probClustVal(self, k, i, val):
        """Find P(j|k)"""
        cj = len([x for x in self.partition[k] if x[i]==val+1])
        nk = len(self.partition[k])
        return (cj + self.alpha[i][val])/(nk + self.alpha0[i])
    
    def condclusterprob(self, stim, k):
        """Find P(F|k)"""
        pjks = []
        for i in range(len(stim)):
            cj = len([x for x in self.partition[k] if x[i]==stim[i]])
            nk = len(self.partition[k])
            pjks.append( (cj + self.alpha[i][stim[i]-1])/(nk + self.alpha0[i]) )
        return np.product( pjks )
        
    
    def posterior(self, stim):
        """Find P(k|F) for each cluster"""
        pk = np.zeros( len(self.partition) )
        pFk = np.zeros( len(self.partition) )
        
        # existing clusters:
        for k in range(len(self.partition)):
            pk[k] = self.c * len(self.partition[k])/ ((1-self.c) + self.c * self.N)
            if len(self.partition[k])==0: # case of new cluster
                pk[k] = (1-self.c) /(( 1-self.c ) + self.c * self.N)
            pFk[k] = self.condclusterprob( stim, k)
        
        # put it together
        pkF = (pk*pFk) / sum( pk*pFk )
        
        return pkF
    
    def stimulate(self, stim):
        """Argmax of P(k|F) + P(0|F)"""
        winner = np.argmax( self.posterior(stim) )
        #print("Stim: ", stim)
        #print("Partition: ", self.partition)
        #print(self.posterior(stim))
        
        if len(self.partition[winner]) == 0:
            self.partition.append( [] )
        self.partition[winner].append(stim)
        #print(len(self.partition))
        self.N += 1
    
    def query(self, stimulus):
        """Queried value should be -1."""
        qdim = -1
        for i in range(len(stimulus)):
            if stimulus[i] < 0:
                if qdim != -1:
                    pass
                qdim = i
        
        self.N = sum([len(x) for x in self.partition])
        
        pkF = self.posterior(stimulus)
        pkF = pkF[:-1] / sum(pkF[:-1])
        
        pjF = np.array( [sum( [ pkF[k] * self.probClustVal(k, qdim, j) \
                for k in range(len(self.partition)-1)] ) 
                for j in range(len( self.alpha[qdim] ))] )
        
        return pjF / sum(pjF)

def testlocalmapD(alpha,c):

    stims = pd.read_csv('X.csv')
    stims = list(np.array(stims))

    for _ in range(1):
        model = dLocalMAP([c,alpha])
        
        #shuffling the data set 
        shuffle(stims)
        for s in stims:
            model.stimulate(s)
        #print(model.partition)
        
        #reading the test data 
        test = pd.read_csv('y.csv',header=None)
        test[2] = -1
        test = np.array(test)
        t = []
        for i in test:
            t.append(np.argmax(model.query( i ))+1)
            stims.append(i+[np.argmax(model.query( i ))+1])
    return t


In [6]:
#trying for differnet parameters
for i in range(5): #c = 0.1 and alpha[3] = [0.7,0.3,0.6]
    print(testlocalmapD([np.ones(100),np.ones(100),[0.7,0.3,0.6]] ,0.1))

[2, 2, 3, 2, 3, 3, 3, 2, 2, 3]
[2, 2, 3, 2, 3, 3, 3, 2, 2, 3]
[2, 2, 3, 2, 3, 3, 3, 2, 3, 3]
[2, 2, 3, 2, 3, 3, 3, 2, 2, 3]
[3, 2, 3, 2, 3, 3, 3, 2, 2, 3]


In [7]:
for i in range(5): #c = 0.3 and alpha[3] = [0.7,0.3,0.6]
    print(testlocalmapD([np.ones(100),np.ones(100),[0.5,0.4,0.5]] ,0.4))

[2, 2, 1, 2, 2, 3, 2, 2, 2, 1]
[2, 2, 1, 2, 2, 3, 2, 2, 2, 1]
[2, 2, 1, 2, 2, 3, 2, 2, 2, 1]
[2, 2, 1, 2, 2, 3, 2, 2, 2, 1]
[2, 2, 1, 2, 2, 3, 2, 2, 2, 1]


In [8]:
for i in range(5): #c = 0.3 and alpha[3] = [0.7,0.3,0.6]
    print(testlocalmapD([np.ones(100),np.ones(100),[0.7,0.3,0.6]] ,0.3))

[2, 2, 1, 2, 2, 3, 2, 2, 2, 1]
[2, 2, 1, 2, 2, 3, 2, 2, 2, 1]
[2, 2, 1, 2, 2, 3, 2, 2, 2, 1]
[2, 2, 1, 2, 2, 3, 2, 2, 2, 1]
[2, 2, 3, 2, 2, 3, 2, 2, 2, 3]


In [9]:
for i in range(5): #c = 0.0001 alpha[2] = [0.5,0.3,0.6]
    print(testlocalmapD([np.ones(100)/100,np.ones(100)/100,[0.5,0.3,0.6]] ,0.0001))

[2, 2, 3, 2, 2, 3, 2, 2, 2, 3]
[2, 2, 3, 2, 2, 3, 2, 2, 2, 3]
[2, 2, 3, 2, 2, 3, 2, 2, 2, 3]
[2, 2, 3, 2, 2, 3, 2, 2, 2, 3]
[2, 2, 3, 2, 2, 3, 2, 2, 2, 3]


In [10]:
pred_y = testlocalmapD([np.ones(100)/100,np.ones(100)/100,[0.5,0.3,0.6]] ,0.0001)
y = pd.DataFrame(y_test)
y[2] = pred_y
y.to_csv('Q2_Y.csv',index= None)
y

,0,1,2
0,74,67,2
1,69,63,2
2,92,81,3
3,64,61,2
4,66,84,2
5,76,68,3
6,61,58,2
7,64,76,2
8,68,66,2
9,34,61,3


c = coupling probabliltiy \
alpha0 = strength of beleif of priors
\
In RMC we assume that the category is also the feature of that data point the category labels are learned by clustering the stimuli together using Bayesian inference. In RMC there no fixed number of cluster , clusters are dependent on coupling probability. So by greater value of c we were getting only one label, by decresing value of c we were getting different labels 




# Q3

In [11]:
#running the gcm for 10 trails and shuffling the input data
for i in range(10):
    #shuffling the train data 
    np.random.shuffle(x_train)
    print('for trails',i,' - ',predict(x_train,y_test,a,g))

for trails 0  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 1]
for trails 1  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 1]
for trails 2  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 1]
for trails 3  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 1]
for trails 4  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 1]
for trails 5  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 1]
for trails 6  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 1]
for trails 7  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 1]
for trails 8  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 1]
for trails 9  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 1]


For RMC tried different parameters to prove that RMC has data exchangeability.

In [12]:
#running the rmc for 10 trails data is being shuffled inside the funtion
#one of the parameters for which the data is consistent
for i in range(10):
     print('for trails',i,' - ',testlocalmapD([np.ones(100)/100,np.ones(100)/100,[0.5,0.3,0.6]] ,0.0001))

for trails 0  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 3]
for trails 1  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 3]
for trails 2  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 3]
for trails 3  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 3]
for trails 4  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 3]
for trails 5  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 3]
for trails 6  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 3]
for trails 7  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 3]
for trails 8  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 3]
for trails 9  -  [2, 2, 3, 2, 2, 3, 2, 2, 2, 3]


The above results show that the order in which we enter the data does not matter.